## COMP257 Data Science Portfolio Part 2
### Student Name: Justin Lam 
### Student ID: 45197083


### Import Relevent Packages

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
import matplotlib.pylab as plt
from datetime import date
from datetime import timedelta
%matplotlib inline

### Loading the data

In [3]:
df = pd.read_csv('energydata_complete.csv')
df.head() 

FileNotFoundError: [Errno 2] File b'energydata_complete.csv' does not exist: b'energydata_complete.csv'

### Expoloratary Data Analysis - EDA

1)  Data Type For Each Attribute. From the output below, it shows that most of the data are 'float64 - floating point' while appliances and lights are interger and finally the data is an object. It is important to find out the data type of attributes as we may need to convert some of the data to perform linear/logistic regression. 

In [ ]:
df.dtypes

From the output above, it shows most of the datatype of variables are float64 and Appliances and lights are intergers. Additionally, we also noticed that date is object therefore we may need to use a command to convert the date into a time series. 

2) Use the commend .describe() to obtain the descriptive statistics of the data. This includes the count, mean, standard deviation, minimum/maximum and the quartiles. 

Descriptive Statistics for temperature. 

In [ ]:
dftemp = df[['T1','T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9']]
dftemp.describe()

Descriptive Statistics for humanlity. 

In [ ]:
dfhum = df[['RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7', 'RH_8', 'RH_9']]
dfhum.describe()

Desriptive Statistics for Weather. 

In [ ]:
dfwea = df[['T_out', 'Press_mm_hg', 'Windspeed', 'Visibility', 'Tdewpoint', 'rv1', 'rv2']]
dfwea.describe()

3) We can also conduct correlation coefficients to find out the relationship between variables. This can be achieved by using the .corr command. 

In [ ]:
df.corr()

#### Figure 7:  (Top) Appliances energy consumption measurement for the whole period.  (Bottom) A closer look at the first week of date

The first step is to recreate the date into index - otherwise we cannot create a plot. Recall the first step of EDA, it shows the default datatype of date is an object. 

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
date = df['date']
Appliances = df['Appliances']
plt.plot(date, Appliances)
plt.xlabel('Time')
plt.ylabel('Appliances Wh')

In [ ]:
week1 = df[df.index < df.index[0] + timedelta(weeks=1)]
week1.plot(kind = 'line', y = 'Appliances', figsize=(14,5))
plt.suptit('Energy Use in the first week')

In [ ]:
fig7b = pd.date_range(start='2016-01-11', end='2016-01-18', freq='D')
date.dtype

####  Fiqure 8: Appliances energy consumption distribution. Top: Histogram shows the frequency of energy consumption in the interval (bar width), and the boxplot shows the location of the median with the black line. 

In [ ]:
bins = 40
plt.hist(df['Appliances'], facecolor='blue', alpha=0.5)
plt.xlabel('Appliances Wh')
plt.ylabel('Frequency')
plt.title('Appliances energy consumption distribution - Histogram')

In [ ]:
df.boxplot(column = 'Appliances', vert=False)
plt.xlabel('Appliances Wh')
plt.title('Appliances energy consumption distribution - Box Plot')

#### Figure 9: Pair plots. Relationship between the energy consumption with lights T1, RH_1, T2, RH_2, T3, RH_3. T1 and RH_1 correspond to the kitchen conditions. T2 and RH_2 correspond to the living room conditions.

In [ ]:
fig9 = df[['T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3']]
sns.pairplot(fig9)

Based on the pairplot in the papaer, it also inculudes the correlation between the variables above however since Python's pariplot does not include correlation. The code will below will conduct correlation.

In [ ]:
fig9.corr()

#### Figure 10: Hourly energy consumption of appliances heat maps for consecutive weeks.

Multiple Regression - Using all the varaibles that is in the complete dataset.

In [ ]:
reg = linear_model.LinearRegression()
x = df[['lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'T4', 'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8', 'RH_8', 'T9', 'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed', 'Visibility', 'Tdewpoint', 'rv1', 'rv2']]
y = df['Appliances']
reg.fit(x, y)
print("y = x *", reg.coef_, "+", reg.intercept_)

Calculate the residuals

In [ ]:
predicted = reg.predict(x)
print("MSE:", mean_squared_error(y, predicted))
print("R^2:", r2_score(y, predicted))

#### Set up RFE (Recursive Feature Elimination) 
- Select all the variables and form an array. 
- Define the X (Independent) and Y (Dependent Variables)
- We manually split the data into training and testing data. Based on the paper, the authors have used 75% of the data as training and 25% to be testing data. 
- Import LinearRegression, RFE and Mean_Squared_Error (MSE) and R^2 as our metrics for regreesion. Both MSE and R^2 will determine how well our model fits. 
- Note that, the complete dataset (energy_completed.csv) is used in this analysis. On the other hand, the authors have used the separate datasets (both training and test) to conduct RFE - as a result there may be a sight variation of the result. 

In [ ]:
feature_cols = np.array(['lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'T4', 'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8', 'RH_8', 'T9', 'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed', 'Visibility', 'Tdewpoint', 'rv1', 'rv2'])
X = df[feature_cols]
y = df['Appliances']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error, r2_score


estimator = LinearRegression()
selector = RFE(estimator, 32) #We asked RFE to select 32 features. 
selector = selector.fit(X, y)

supp = selector.get_support()
print("Selected features:", feature_cols[supp])
print("Coeffs:", selector.estimator_.coef_)
# test the model
predicted = selector.predict(X)
print("MSE:", mean_squared_error(y, predicted))
print("R^2:", r2_score(y, predicted))

#### Interpretation of results. 
- Based on the R^2 of the model, it shows that 16.4% of the data are explained by using our data. In other words, regression is prehaps not the best technique to use in this dataset. Additionally, we can also look at the coefficients of the features to determine how RFE ranks them in regards of significance. 

#### Further Regression Analysis - Using Training and Testing Data

In [ ]:
train = pd.read_csv('training.csv')
test = pd.read_csv('testing.csv')
train.shape, test.shape

Load both training and testing dataset and check the number of observations. 

In [ ]:
train.info()
train.head()

By looking at the data type of each variables, it shows that variables 'WeekStatus' and 'Day_of_week' are categorical variables.Therefore we might need to convert them into a dummy variable (either 0 for weekday or 1 weekend) to conduct regression analysis for both training data set. 

In [ ]:
train1 = pd.get_dummies(train, columns=['WeekStatus'])
train1.info()

In [ ]:
reg1 = linear_model.LinearRegression()
x_train = train1[['lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'T4', 'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8', 'RH_8', 'T9', 'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed', 'Visibility', 'Tdewpoint', 'rv1', 'rv2', 'NSM', 'WeekStatus_Weekday', 'WeekStatus_Weekend']]
y_train = train1['Appliances']
reg1.fit(x_train,y_train)
print("y = x *", reg1.coef_, "+", reg1.intercept_)

In [ ]:
predicted2 = reg1.predict(x_train)
print("MSE:", mean_squared_error(y_train, predicted2))
print("R^2:", r2_score(y_train, predicted2))

We conduct regression by using the testing data (which the variable 'Day of Week'has already been converted in a dummy variable). The indpendent variables of the regression will be of the variables provided while we are using these variables to predict the level of energy consumption of appliances. 

In [ ]:
reg2 = linear_model.LinearRegression()
x_test = test[['lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'T4', 'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8', 'RH_8', 'T9', 'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed', 'Visibility', 'Tdewpoint', 'rv1', 'rv2', 'NSM', 'WeekStatus_Weekday', 'WeekStatus_Weekend']]
y_test = test['Appliances']
reg2.fit(x_test,y_test)
print("y = x *", reg2.coef_, "+", reg2.intercept_)

Similarly, we also to conduct the MSE and R^2 of the model. By observing the results, it appears that both of the numbers are similar to the ones in the paper - which is a good sign.

In [ ]:
predicted3 = reg1.predict(x_test)
print("MSE:", mean_squared_error(y_test, predicted3))
print("R^2:", r2_score(y_test, predicted3))